In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras.models import Sequential
from keras.utils.np_utils import to_categorical 
from keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')

In [ ]:
train.head()

In [ ]:
train.shape

Data Preprocessing

In [ ]:
labels = train['label']
labels = np.array(labels)
labels

In [ ]:
del train['label']
train.shape

In [ ]:
train = train / 255
test = test / 255

In [ ]:
print(train.shape)
print(labels.shape)
print(test.shape)

In [ ]:
# Reshape the data
train = np.array(train).reshape(-1, 28, 28)
train.shape

In [ ]:
# Add dimension
train = np.expand_dims(train,axis=-1)
labels = np.expand_dims(labels,axis=-1)

In [ ]:
train.shape

In [ ]:
# labels category
labels = to_categorical(labels, num_classes = 10)

In [ ]:
# Check the shape
print(train.shape)
print(labels.shape)

In [ ]:
# Convert test data
test = test.to_numpy().reshape(-1,28,28)
test = np.expand_dims(test,axis=-1)

test.shape

In [ ]:
# Split data and labels for train and test datasets
X_train, X_test, y_train, y_test = train_test_split(train, labels, test_size = 0.17)

In [ ]:
# Check the shape for train data
print(X_train.shape)
print(y_train.shape)

In [ ]:
# Check the shape for test data
print(X_test.shape)
print(y_test.shape)

In [ ]:
# Build up model - Sequential (Convolutional Model)
model = tf.keras.Sequential([])
model.add(tf.keras.layers.Conv2D(32, (3, 3), padding = 'same', activation='relu', input_shape=(28, 28, 1)))
model.add(tf.keras.layers.MaxPooling2D(2, 2)),
model.add(tf.keras.layers.Conv2D(64, (3, 3), padding = 'same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(2, 2)),
model.add(tf.keras.layers.Conv2D(128, (3, 3), padding = 'same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(2, 2)),
model.add(tf.keras.layers.Conv2D(128, (3, 3), padding = 'same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(2, 2)),
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#Add ImageDataGenerator
datagen = ImageDataGenerator(rotation_range=20, zoom_range=0.2)
datagen.fit(X_train)

In [ ]:
# Fit model
#model.fit(X_train, y_train, validation_data = (X_test,y_test), epochs=10)
model.fit_generator(datagen.flow(X_train, y_train, batch_size=32),
                    steps_per_epoch=len(X_train)/32, epochs=10)

In [ ]:
# Evaluation
model.evaluate(X_test,y_test)

In [ ]:
# Predict data
predictions = model.predict(test)
y_pred = np.argmax(predictions, axis=1)

In [ ]:
# Show some prediction
plt.figure(figsize=(5,5))

for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(test[i])
    plt.xticks([])
    plt.yticks([])
    plt.xlabel(y_pred[i])

In [ ]:
submission =  pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')

In [ ]:
# Save data
submission['Label'] = pd.DataFrame(y_pred)

In [ ]:
submission.to_csv('/kaggle/working/MySubmissions.csv', index = False)